# 채팅 완료 요청을 스트리밍하는 방법

기본적으로 완료를 요청하면 전체 완료가 생성된 후 단일 응답으로 다시 전송됩니다.

긴 완료를 생성하는 경우 응답을 기다리는 데 몇 초가 걸릴 수 있습니다.

응답을 더 빨리 받으려면 완료가 생성되는 동안 '스트리밍'할 수 있습니다. 이렇게 하면 전체 완료가 끝나기 전에 완료의 시작 부분을 인쇄하거나 처리할 수 있습니다.

완료를 스트리밍하려면 채팅 완료 또는 완료 엔드포인트를 호출할 때 `stream=True`를 설정합니다. 그러면 [데이터 전용 서버 전송 이벤트](https://developer.mozilla.org/en-US/docs/Web/API/Server-sent_events/Using_server-sent_events#event_stream_format)로 응답을 다시 스트리밍하는 개체가 반환됩니다. `message` 필드 대신 `delta` 필드에서 청크를 추출합니다.

## 단점

프로덕션 애플리케이션에서 `stream=True`를 사용하면 부분 완료를 평가하기가 더 어려울 수 있으므로 완료 내용의 조정이 더 어려워집니다.
## 예제 코드

아래에서 이 노트북은 다음을 보여줍니다.
1. 일반적인 채팅 완료 응답의 모양
2. 스트리밍 채팅 완료 응답의 모양
3. 채팅 완료를 스트리밍하여 절약되는 시간

In [1]:
# 가져오기
import easyllm  # API 호출용
import time  # API 호출 시간 측정용

### 1. 일반적인 채팅 완료 응답의 모양

일반적인 ChatCompletions API 호출을 사용하면 응답이 먼저 계산된 다음 한 번에 모두 반환됩니다.

In [3]:
from easyllm.clients import huggingface

# 프롬프트 빌더를 llama2로 설정합니다.
huggingface.prompt_builder = "llama2"

# 요청이 전송되기 전 시간을 기록합니다.
start_time = time.time()

# 100까지 세는 ChatCompletion 요청을 보냅니다.
response = huggingface.ChatCompletion.create(
    model="meta-llama/Llama-2-70b-chat-hf",
    messages=[
        {'role': 'user', 'content': '100까지 세되 각 숫자 사이에 쉼표를 넣고 줄 바꿈은 하지 마세요. 예: 1, 2, 3, ...'}
    ],
)

# 응답을 받는 데 걸린 시간을 계산합니다.
response_time = time.time() - start_time

# 시간 지연 및 수신된 텍스트를 인쇄합니다.
print(f"요청 후 {response_time:.2f}초 만에 전체 응답 수신")
print(f"전체 응답 수신:\n{response}")


요청 후 0.12초 만에 전체 응답 수신
전체 응답 수신:
{'id': 'hf-JhxbFCGVUW', 'object': 'chat.completion', 'created': 1691129826, 'model': 'meta-llama/Llama-2-70b-chat-hf', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': ' 물론입니다! 여기 있습니다:\n\n1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100'}, 'finish_reason': 'eos_token'}], 'usage': {'prompt_tokens': 25, 'completion_tokens': 400, 'total_tokens': 425}}


답변은 `response['choices'][0]['message']`로 추출할 수 있습니다.

답변 내용은 `response['choices'][0]['message']['content']`로 추출할 수 있습니다.

In [4]:
reply = response['choices'][0]['message']
print(f"추출된 답변: \n{reply}")

reply_content = response['choices'][0]['message']['content']
print(f"추출된 내용: \n{reply_content}")


추출된 답변: 
{'role': 'assistant', 'content': ' 물론입니다! 여기 있습니다:\n\n1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100'}
추출된 내용: 
 물론입니다! 여기 있습니다:

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100


### 2. 채팅 완료를 스트리밍하는 방법

스트리밍 API 호출을 사용하면 응답이 [이벤트 스트림](https://developer.mozilla.org/en-US/docs/Web/API/Server-sent_events/Using_server-sent_events#event_stream_format)을 통해 증분적으로 청크로 전송됩니다. Python에서는 `for` 루프를 사용하여 이러한 이벤트를 반복할 수 있습니다.

어떻게 보이는지 살펴보겠습니다.

In [5]:
from easyllm.clients import huggingface


huggingface.prompt_builder = "llama2"

# ChatCompletion 요청
response = huggingface.ChatCompletion.create(
    model="meta-llama/Llama-2-70b-chat-hf",
    messages=[
        {'role': 'user', 'content': "1+1은 무엇인가요? 한 단어로 답해주세요."}
    ],
    stream=True  # 이번에는 stream=True로 설정합니다.
)

for chunk in response:
    print(chunk)

{'id': 'hf--G5Fhg3YMu', 'object': 'chat.completion.chunk', 'created': 1691129830, 'model': 'meta-llama/Llama-2-70b-chat-hf', 'choices': [{'index': 0, 'delta': {'role': 'assistant'}}]}
{'id': 'hf--G5Fhg3YMu', 'object': 'chat.completion.chunk', 'created': 1691129830, 'model': 'meta-llama/Llama-2-70b-chat-hf', 'choices': [{'index': 0, 'delta': {'content': ' '}}]}
{'id': 'hf--G5Fhg3YMu', 'object': 'chat.completion.chunk', 'created': 1691129830, 'model': 'meta-llama/Llama-2-70b-chat-hf', 'choices': [{'index': 0, 'delta': {'content': ' 둘'}}]}
{'id': 'hf--G5Fhg3YMu', 'object': 'chat.completion.chunk', 'created': 1691129830, 'model': 'meta-llama/Llama-2-70b-chat-hf', 'choices': [{'index': 0, 'delta': {}}]}


위에서 볼 수 있듯이 스트리밍 응답에는 `message` 필드 대신 `delta` 필드가 있습니다. `delta`는 다음과 같은 항목을 포함할 수 있습니다.
- 역할 토큰 (예: `{"role": "assistant"}`)
- 콘텐츠 토큰 (예: `{"content": "\n\n"}`)
- 스트림이 끝나면 아무것도 없음 (예: `{}`)

### 3. 채팅 완료를 스트리밍하여 절약되는 시간

이제 `meta-llama/Llama-2-70b-chat-hf`에게 다시 100까지 세도록 요청하고 얼마나 걸리는지 확인해 보겠습니다.

In [6]:
import time
from easyllm.clients import huggingface

huggingface.prompt_builder = "llama2"

# record the time before the request is sent
start_time = time.time()

# send a ChatCompletion request to count to 100
response = huggingface.ChatCompletion.create(
    model="meta-llama/Llama-2-70b-chat-hf",
    messages=[
        {'role': 'user', 'content': 'Count to 50, with a comma between each number and no newlines. E.g., 1, 2, 3, ...'}
    ],
    stream=True  # again, we set stream=True
)

# create variables to collect the stream of chunks
collected_chunks = []
collected_messages = []
# iterate through the stream of events
for chunk in response:
    chunk_time = time.time() - start_time  # calculate the time delay of the chunk
    collected_chunks.append(chunk)  # save the event response
    chunk_message = chunk['choices'][0]['delta']  # extract the message
    collected_messages.append(chunk_message)  # save the message
    print(f"Message received {chunk_time:.2f} seconds after request: {chunk_message}")  # print the delay and text

# print the time delay and text received
print(f"Full response received {chunk_time:.2f} seconds after request")
full_reply_content = ''.join([m.get('content', '') for m in collected_messages])
print(f"Full conversation received: {full_reply_content}")


요청 후 0.13초 만에 메시지 수신: {'role': 'assistant'}
요청 후 0.13초 만에 메시지 수신: {'content': ' '}
요청 후 0.13초 만에 메시지 수신: {'content': ' 물론'}
요청 후 0.13초 만에 메시지 수신: {'content': '!'}
요청 후 0.13초 만에 메시지 수신: {'content': ' 여기'}
요청 후 0.13초 만에 메시지 수신: {'content': ' 있'}
요청 후 0.13초 만에 메시지 수신: {'content': '습니다'}
요청 후 0.13초 만에 메시지 수신: {'content': ':'}
요청 후 0.13초 만에 메시지 수신: {'content': '\n'}
요청 후 0.13초 만에 메시지 수신: {'content': '\n'}
요청 후 0.13초 만에 메시지 수신: {'content': '1'}
요청 후 0.13초 만에 메시지 수신: {'content': ','}
요청 후 0.13초 만에 메시지 수신: {'content': ' '}
요청 후 0.13초 만에 메시지 수신: {'content': '2'}
요청 후 0.13초 만에 메시지 수신: {'content': ','}
요청 후 0.13초 만에 메시지 수신: {'content': ' '}
요청 후 0.13초 만에 메시지 수신: {'content': '3'}
요청 후 0.13초 만에 메시지 수신: {'content': ','}
요청 후 0.13초 만에 메시지 수신: {'content': ' '}
요청 후 0.13초 만에 메시지 수신: {'content': '4'}
요청 후 0.13초 만에 메시지 수신: {'content': ','}
요청 후 0.13초 만에 메시지 수신: {'content': ' '}
요청 후 0.13초 만에 메시지 수신: {'content': '5'}
요청 후 0.13초 만에 메시지 수신: {'content': ','}
요청 후 0.13초 만에 메시지 수신: {'content': ' '}
요청 후 0.13초 

#### 시간 비교

위 예제에서 두 요청 모두 완전히 완료되는 데 약 3초가 걸렸습니다. 요청 시간은 부하 및 기타 확률적 요인에 따라 달라집니다.

그러나 스트리밍 요청을 사용하면 0.1초 후에 첫 번째 토큰을 수신하고 이후 토큰은 약 0.01~0.02초마다 수신했습니다.